# Sklearn

## Bike Sharing Demand
Задача на kaggle: https://www.kaggle.com/c/bike-sharing-demand

По историческим данным о прокате велосипедов и погодным условиям необходимо оценить спрос на прокат велосипедов.

В исходной постановке задачи доступно 11 признаков: https://www.kaggle.com/c/prudential-life-insurance-assessment/data

В наборе признаков присутсвуют вещественные, категориальные, и бинарные данные. 

Для демонстрации используется обучающая выборка из исходных данных train.csv, файлы для работы прилагаются.

### Библиотеки

In [ ]:
from sklearn import model_selection, linear_model, metrics
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
%pylab inline

### Загрузка данных

In [ ]:
raw_data = pd.read_csv('bike_sharing_demand.csv', header = 0, sep = ',')

In [ ]:
# первые 5 значений
raw_data.head(7)

In [ ]:
# последние 7 значений
raw_data.tail(7)

In [ ]:
# какие категории в столбце  holiday
raw_data.groupby(['holiday']).count()

In [ ]:
# какие категории в столбце workingday
raw_data.groupby(['workingday']).count()

In [ ]:
# какие категории в столбце weather
raw_data.groupby(['weather']).count()

In [ ]:
# Вопрос 1. размерность нашего датасета
# вставьте свой код





***datetime*** - hourly date + timestamp  

***season*** -  1 = spring, 2 = summer, 3 = fall, 4 = winter 

***holiday*** - whether the day is considered a holiday

***workingday*** - whether the day is neither a weekend nor holiday

***weather*** - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
    
***temp*** - temperature in Celsius

***atemp*** - "feels like" temperature in Celsius

***humidity*** - relative humidity

***windspeed*** - wind speed

***casual*** - number of non-registered user rentals initiated

***registered*** - number of registered user rentals initiated

***count*** - number of total rentals

In [ ]:
print(raw_data.shape)

In [ ]:
# миссинговые
raw_data.isnull().values.any()

In [ ]:
raw_data.info()

In [ ]:
# Считайте данные и выведите первые 5 строк
df = pd.read_csv('bikes_rent.csv')
df.head()

Для каждого дня проката известны следующие признаки (как они были указаны в источнике данных):
* _season_: 1 - весна, 2 - лето, 3 - осень, 4 - зима
* _yr_: 0 - 2011, 1 - 2012
* _mnth_: от 1 до 12
* _holiday_: 0 - нет праздника, 1 - есть праздник
* _weekday_: от 0 до 6
* _workingday_: 0 - нерабочий день, 1 - рабочий день
* _weathersit_: оценка благоприятности погоды от 1 (чистый, ясный день) до 4 (ливень, туман)
* _temp_: температура в Цельсиях
* _atemp_: температура по ощущениям в Цельсиях
* _hum_: влажность
* _windspeed(mph)_: скорость ветра в милях в час
* _windspeed(ms)_: скорость ветра в метрах в секунду
* _cnt_: количество арендованных велосипедов (это целевой признак, его мы будем предсказывать)

Итак, у нас есть вещественные, бинарные и номинальные (порядковые) признаки, и со всеми из них можно работать как с вещественными. С номинальныеми признаками тоже можно работать как с вещественными, потому что на них задан порядок. Давайте посмотрим на графиках, как целевой признак зависит от остальных

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 10))
for idx, feature in enumerate(df.columns[:-1]):
    df.plot(feature, "cnt", subplots=True, kind="scatter", ax=axes[idx // 4, idx % 4])

In [ ]:
# 2. Каков характер зависимости числа прокатов от месяца? 

# 2. Укажите один или два признака, от которых число прокатов скорее всего зависит линейно
  

In [ ]:
# Посчитайте корреляции всех признаков, кроме последнего, с последним с помощью метода corrwith:
# features - сюда записываем все ваши фичи.
features = df.iloc[:, :-1]
print(features.corrwith(df['cnt']))

In [ ]:
# Посчитайте попарные корреляции между признаками temp, atemp, hum, windspeed(mph), windspeed(ms) и cnt
# с помощью метода corr:
raw_data.iloc[:, :].corr()

In [ ]:
# ВОПРОС 3. Выведите средние признаков features с помощъю .mean() или .describe()
# вставьте свой код и впишите в форму средние значения по temp, hum



### Предобработка данных

#### Типы признаков

In [ ]:
raw_data.datetime = raw_data.datetime.apply(pd.to_datetime)

In [ ]:
raw_data.info()

In [ ]:
raw_data.head()

In [ ]:
# Вытаскиваем месяц час и год из нашей даты
raw_data['month'] = raw_data.datetime.apply(lambda x : x.month)
raw_data['hour'] = raw_data.datetime.apply(lambda x : x.hour)

In [ ]:
raw_data['year'] = raw_data.datetime.apply(lambda x: x.year).map({2011:0, 2012:1})

In [ ]:
raw_data.head()

#### Обучение и отложенный тест (train и test!)

In [ ]:
train_data = raw_data.iloc[:-1000, :]
hold_out_test_data = raw_data.iloc[-1000:, :]

In [ ]:
train_data.shape, hold_out_test_data.shape

In [ ]:
# Вопрос 4. Какое соотношение у обучающей выборки (train_data) и тестовой выборки(hold_out_test_data) по отношению ко всему датасету?


In [ ]:
train_data.groupby(['year']).count()

In [ ]:
hold_out_test_data.groupby(['year']).count()

In [ ]:
print(raw_data.shape, train_data.shape, hold_out_test_data.shape)

In [ ]:
train_data.datetime.min(), train_data.datetime.max()

In [ ]:
# какие временные промежутки приходятся на обучающую выборку и тестовую выборку
print('train period from {} to {}'.format(train_data.datetime.min(), train_data.datetime.max()))
print('evaluation period from {} to {}'.format(hold_out_test_data.datetime.min(), hold_out_test_data.datetime.max()))

#### Данные и целевая функция

In [ ]:
#обучение
train_labels = train_data['count'].values
train_data = train_data.drop(['datetime', 'count'], axis = 1)

In [ ]:
train_data.shape

In [ ]:
hold_out_test_data.shape

In [ ]:
#тестовая выборка
test_labels = hold_out_test_data['count'].values
test_data = hold_out_test_data.drop(['datetime', 'count'], axis = 1)

In [ ]:
len(test_labels)

In [ ]:
test_data.shape

#### Целевая функция на обучающей выборке и на отложенном тесте

In [ ]:
pylab.figure(figsize = (16, 6))

pylab.subplot(1,2,1)
pylab.hist(train_labels)
pylab.title('train data')

pylab.subplot(1,2,2)
pylab.hist(test_labels)
pylab.title('test data')

#### Числовые признаки (для дальнейшего обучения модели берем только числовые переменные)

In [ ]:
# записываем в numeric_columns числовые переменные
numeric_columns = ['temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'month', 'hour']

In [ ]:
train_data.shape

In [ ]:
# берем только числовые переменные в обучающую выборку
train_data = train_data[numeric_columns]

In [ ]:
# берем только числовые переменные в тестовую выборку
test_data = test_data[numeric_columns]

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
test_data.shape

### Модель

In [ ]:
from sklearn.linear_model import LinearRegression, SGDRegressor

In [ ]:
# Постройте базовую линейную регрессию с помощью LinearRegression и метода fit
# вставьте свой код
model = 

In [ ]:
# Обучите вашу модель
model.fit(train_data, train_labels)

In [ ]:
# Запишите в model_predictions ваши предсказания по test_data
# вставьте свой код
model_predictions = 

In [ ]:
# ПОлучите метрику МАЕ 
# Вопрос 5. Какая у вас выходит средняя абсолютная ошибка MAE
metrics.mean_absolute_error(test_labels, model_predictions)

In [ ]:
# Распечатайте первые 10 предсказаний 
model_predictions[:10]

In [ ]:
# Распечатайте первые 10 фактических значений
# Хорошо ли мы предксазываем спрос на велосипеды? 
test_labels[:10]
